**Задача:**

Определить текущий уровень потребительской лояльности, или NPS (от англ. Net Promoter Score), среди клиентов из России. 

**Заказчик:**

Большая телекоммуникационная компания, которая оказывает услуги на территории всего СНГ.

**Описание данных**
- Таблица `user`. Содержит основную информацию о клиентах.
    - `user_id` —	Идентификатор клиента, первичный ключ таблицы
    - `lt_day` —	Количество дней «жизни» клиента
    - `age`—	Возраст клиента в годах
    - `gender_segment` —	Пол клиента (1 – женщина, 0 – мужчина)
    - `os_name` —	Тип операционной системы
    - `cpe_type_name` —	Тип устройства
    - `location_id` —	Идентификатор домашнего региона клиента, внешний ключ, отсылающий к таблице location
    - `age_gr_id` —	Идентификатор возрастного сегмента клиента, внешний ключ, отсылающий к таблице age_segment
    - `tr_gr_id` —	Идентификатор сегмента клиента по объёму потребляемого трафика в месяц, внешний ключ, отсылающий к таблице traffic_segment
    - `lt_gr_id` —	Идентификатор сегмента клиента по количеству месяцев «жизни», внешний ключ, отсылающий к таблице lifetime_segment
    - `nps_score` — Оценка клиента в NPS-опросе (от 1 до 10)
    
- Таблица `location`. Справочник территорий, в которых телеком-компания оказывает услуги.
    - `location_id` —	Идентификатор записи, первичный ключ
    - `country` —	Страна
    - `city` —	Город
    
- Таблица `age_segment`. Данные о возрастных сегментах клиентов.
    - `age_gr_id` —	Идентификатор сегмента, первичный ключ
    - `bucket_min` —	Минимальная граница сегмента
    - `bucket_max` — 	Максимальная граница сегмента
    - `title` —	Название сегмента
    
- Таблица `traffic_segment`. Данные о выделяемых сегментах по объёму потребляемого трафика.
    - `tr_gr_id` —	Идентификатор сегмента, первичный ключ
    - `bucket_min` —	Минимальная граница сегмента
    - `bucket_max` —	Максимальная граница сегмента
    - `title` —	Название сегмента
    
- Таблица `lifetime_segment`. Данные о выделяемых сегментах по количеству месяцев «жизни» клиента — лайфтайму.
    - `lt_gr_id` —	Идентификатор сегмента, первичный ключ
    - `bucket_min` —	Минимальная граница сегмента
    - `bucket_max` —	Максимальная граница сегмента
    - `title` —	Название сегмента


**Этапы исследования**

[Шаг 1. Подключение к базе](#connect)

[Шаг 2. Выгрузка данных](#data)
Задача: Преобразовывать данные без Python. Только с помощью SQL-запросов. 
Выгрузить поля:
- `user_id` —	Идентификатор клиента
- `lt_day`	 — Количество дней «жизни» клиента
- `is_new`	 — Поле хранит информацию о том, является ли клиент новым
- `age`	 — Возраст
- `gender_segment`	 — Пол (для удобства работы с полем преобразуйте значения в текстовый вид)
- `os_name`	 — Тип операционной системы
- `cpe_type_name`	 — Тип устройства
- `country`	 — Страна проживания
- `city`	 — Город проживания
- `age_segment`	 — Возрастной сегмент
- `traffic_segment`	 — Сегмент по объёму потребляемого трафика
- `lifetime_segment`	 — Сегмент по количеству дней «жизни»
- `nps_score`	 — Оценка клиента в NPS-опросе
- `nps_group`	 — Поле хранит информацию о том, к какой группе относится оценка клиента в опросе

[Шаг 3. Создание дашборда в Tableau](#dashbord)

Построить дашборд, который представит информацию о текущем уровне NPS среди клиентов и покажет, как этот уровень меняется в зависимости от пользовательских признаков. 

Из дашборда должно быть понятно, какие группы пользователей участвовали в опросе.

[Шаг 4. Создание презентации с ответами на следующие вопросы:](#presentation)
- Как распределены участники опроса по возрасту и полу? Каких пользователей больше: новых или старых? Пользователи из каких городов активнее участвовали в опросе?
- Какие группы пользователей наиболее лояльны к сервису? Какие менее?
- Какой общий NPS среди всех опрошенных?
- Как можно описать клиентов, которые относятся к группе cторонников (англ. promoters)?


<a id="connect"></a> 
<h2>Шаг. Подключение к базе</h2>

In [1]:
import os
import pandas as pd
import numpy as np

from sqlalchemy import create_engine

In [2]:
# путь к БД на компьютере 
path_to_db_local = 'telecomm_csi.db'
# путь к БД на платформе
path_to_db_platform = '/datasets/telecomm_csi.db'
# итоговый путь к БД
path_to_db = None
# если путь на компьютере ведёт к БД, то он становится итоговым
if os.path.exists(path_to_db_local):
    path_to_db = path_to_db_local
# иначе: если путь на платформе ведёт к БД, то он становится итоговым
elif os.path.exists(path_to_db_platform):
    path_to_db = path_to_db_platform
# иначе выводится сообщение о том, что файл не найден
else:
    raise Exception('Файл с базой данных SQLite не найден!')
# если итоговый путь не пустой
if path_to_db:
        # то создаём подключение к базе
    engine = create_engine(f'sqlite:///{path_to_db}', echo=False)

<a id="data"></a> 
<h2>Шаг. Выгрузка данных</h2>

In [3]:
# Проверка значений nps_score на диапазон от 1 до 10
query_nps_score = """
SELECT *
FROM user
WHERE nps_score < 1 OR nps_score > 10;
"""

In [4]:
nps_out_of_range = pd.read_sql(query_nps_score, engine)
if not nps_out_of_range.empty:
    print("Есть значения nps_score вне диапазона 1-10")
else:
    print("Все значения nps_score находятся в диапазоне 1-10")

Все значения nps_score находятся в диапазоне 1-10


In [5]:
# Проверка дубликатов в поле city (учитывая регистр и пробелы)
query_duplicates_city = """
SELECT LOWER(TRIM(city)) AS normalized_city, COUNT(*)
FROM location
GROUP BY normalized_city
HAVING COUNT(*) > 1;
"""

In [6]:
duplicates_city = pd.read_sql(query_duplicates_city, engine)
if not duplicates_city.empty:
    print("Есть потенциальные дубликаты городов:")
    print(duplicates_city.head())
else:
    print("Нет неявных дубликатов городов")

Нет неявных дубликатов городов


In [7]:
# Проверка отрицательных значений в age
query_negative_age = """
SELECT *
FROM user
WHERE age < 0;
"""

In [8]:
negative_age = pd.read_sql(query_negative_age, engine)
if not negative_age.empty:
    print("Есть отрицательные значения в поле age")
else:
    print("Нет отрицательных значений в поле age")

Нет отрицательных значений в поле age


In [9]:
query = """
    SELECT u.user_id,
       u.lt_day,
       CASE
           WHEN u.lt_day <= 365 THEN 'TRUE'
           ELSE 'FALSE'
       END AS is_new,
       u.age,
       CASE 
           WHEN u.gender_segment = 1 THEN 'женщина'
           WHEN u.gender_segment = 0 THEN 'мужчина'
           ELSE 'другой'
       END AS gender_segment,
       u.os_name,
       u.cpe_type_name,
       l.country,
       l.city,
       SUBSTRING (a.title, 3 )AS age_segment,
       SUBSTRING (t.title,3) AS traffic_segment,
       SUBSTRING (lf.title,3) AS lifetime_segment,
       u.nps_score,
       CASE 
           WHEN u.nps_score >= 9 THEN 'сторонники'
           WHEN u.nps_score >= 7 THEN 'нейтралы'
           ELSE 'критики'
       END AS nps_group
FROM user AS u
JOIN location AS l ON u.location_id = l.location_id
JOIN age_segment AS a ON u.age_gr_id = a.age_gr_id
JOIN traffic_segment AS t ON u.tr_gr_id = t.tr_gr_id
JOIN lifetime_segment AS lf ON u.lt_gr_id = lf.lt_gr_id;
    """

In [10]:
# создаём датафрейм по данным запроса
df = pd.read_sql(query, engine)
df.head(3)

,user_id,lt_day,is_new,age,gender_segment,os_name,cpe_type_name,country,city,age_segment,traffic_segment,lifetime_segment,nps_score,nps_group
0,A001A2,2320,FALSE,45.0,женщина,ANDROID,SMARTPHONE,Россия,Уфа,45-54,1-5,36+,10,сторонники
1,A001WF,2344,FALSE,53.0,мужчина,ANDROID,SMARTPHONE,Россия,Киров,45-54,1-5,36+,10,сторонники
2,A003Q7,467,FALSE,57.0,мужчина,ANDROID,SMARTPHONE,Россия,Москва,55-64,20-25,13-24,10,сторонники


In [11]:
df.to_csv('telecomm_csi_tableau.csv', index=False)

<a id="dashbord"></a> 
<h2>Шаг. Создание дешборда</h2>

https://public.tableau.com/views/2_17182067598550/NPS_3?:language=en-US&publish=yes&:sid=&:display_count=n&:origin=viz_share_link

<a id="presentation"></a> 
<h2>Шаг. Создание презентации</h2>

https://public.tableau.com/shared/MF824FQX9?:display_count=n&:origin=viz_share_link